# AI Unit Test Generator (JavaScript & TypeScript)

An AI assistant that generates comprehensive unit tests when you provide JavaScript or TypeScript functions or classes. Paste your code, pick a model, and generate runnable Jest tests. Requires Node.js and npm.

In [1]:
import os
import re
import subprocess
import tempfile
import shutil
from pathlib import Path

from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr


In [2]:
MODEL_GPT = 'gpt-4.1-mini'
MODEL_LLAMA = 'kimi-k2.5:cloud'
MODEL_CLAUDE = 'claude-sonnet-4-5-20250929'

In [3]:
load_dotenv(override=True)

openai_api_key = os.getenv("OPENAI_API_KEY")
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (and this is optional)")
models = ["gpt", "claude", "kimi"]


OpenAI API Key exists and begins sk-proj-
Anthropic API Key not set (and this is optional)


In [4]:
openai = OpenAI()

anthropic_url = "https://api.anthropic.com/v1/"
OLLAMA_BASE_URL = "http://localhost:11434/v1"

anthropic = OpenAI(api_key=anthropic_api_key, base_url=anthropic_url)
ollama = OpenAI(api_key="ollama", base_url=OLLAMA_BASE_URL)

openai = OpenAI()

In [5]:
clients = {"gpt": openai, "claude": anthropic, "kimi": ollama, }

In [6]:
SYSTEM_PROMPT_JS = """You are an expert at writing unit tests for JavaScript code.
Generate comprehensive Jest unit tests for the given functions or classes.

Rules:
- Use Jest. Use expect() and toBe(), toEqual(), toThrow(), etc.
- Respond ONLY with JavaScript code. No markdown fences, no prose outside comments.
- Output ONLY the test file. The source is in a separate file named "module.js" (export your functions/classes).
- Import from './module.js' (use ES modules: import { fn } from './module.js').
- Cover: normal cases, edge cases (empty, zero, null, undefined), and error cases.
- Mock external dependencies (fetch, fs, etc.) with jest.fn() or jest.mock().
- Add a brief comment before each test describing what it checks.
- Keep tests deterministic.
"""

SYSTEM_PROMPT_TS = """You are an expert at writing unit tests for TypeScript code.
Generate comprehensive Jest unit tests for the given functions or classes.

Rules:
- Use Jest. Use expect() and toBe(), toEqual(), toThrow(), etc.
- Respond ONLY with TypeScript code. No markdown fences, no prose outside comments.
- Output ONLY the test file. The source is in a separate file named "module.ts" (export your functions/classes).
- Import from './module' (use ES modules: import { fn } from './module').
- Cover: normal cases, edge cases (empty, zero, null, undefined), and error cases.
- Mock external dependencies (fetch, fs, etc.) with jest.fn() or jest.mock().
- Add a brief comment before each test describing what it checks.
- Keep tests deterministic.
- Include proper TypeScript types where helpful.
"""

def user_prompt(code: str, lang: str) -> str:
    ext = "ts" if lang == "typescript" else "js"
    return f"""Generate Jest unit tests for this {lang.upper()} code.
Output ONLY the test file content. The source will be in module.{ext}. Import from './module'.
Respond only with {lang} code, no markdown.

{lang.upper()} code to test:

{code}
"""

In [ ]:
MODEL_IDS = {"gpt": MODEL_GPT, "claude": MODEL_CLAUDE, "kimi": MODEL_LLAMA}

In [8]:
def extract_code(reply: str, lang: str) -> str:
    """Strip markdown code fences from model output."""
    reply = reply.strip()
    for pattern in (r"```(?:javascript|typescript|js|ts)?\s*", r"```\s*"):
        reply = re.sub(f"^{pattern}", "", reply)
        reply = re.sub(f"{pattern}$", "", reply)
    return reply.strip()


def generate_unit_tests(model: str, code: str, lang: str) -> str:
    """Generate unit tests for the given JS/TS code using the selected model."""
    if not code or not code.strip():
        return "Please paste some JavaScript or TypeScript code (functions or classes) first."
    client = clients.get(model)
    if not client:
        return f"Model '{model}' not available. Add API keys to .env."
    system = SYSTEM_PROMPT_TS if lang == "typescript" else SYSTEM_PROMPT_JS
    model_id = MODEL_IDS.get(model, model)
    try:
        response = client.chat.completions.create(
            model=model_id,
            messages=[
                {"role": "system", "content": system},
                {"role": "user", "content": user_prompt(code, lang)},
            ],
        )
        reply = response.choices[0].message.content or ""
        return extract_code(reply, lang)
    except Exception as e:
        return f"Error: {e}"

In [9]:
def run_tests(source_code: str, test_code: str, lang: str) -> str:
    """Create temp project, write source + tests, run Jest."""
    import json

    if not source_code or not source_code.strip():
        return "No source code to run."
    if not test_code or not test_code.strip():
        return "No test code to run. Generate tests first."

    ext = "ts" if lang == "typescript" else "js"
    tmpdir = Path(tempfile.mkdtemp())
    try:
        pkg = {
            "type": "module",
            "scripts": {"test": "node --experimental-vm-modules node_modules/jest/bin/jest.js"},
            "devDependencies": {"jest": "^29.7.0"},
        }
        if lang == "typescript":
            pkg["devDependencies"].update({"ts-jest": "^29.2.0", "typescript": "^5.0.0"})
        (tmpdir / "package.json").write_text(json.dumps(pkg))

        transform = ',"transform":{"^.+\\.tsx?$":"ts-jest"}' if lang == "typescript" else ""
        jest_config = f'export default {{ testEnvironment: "node"{transform}, testMatch: ["**/*.test.{ext}"] }};'
        (tmpdir / "jest.config.js").write_text(jest_config)
        (tmpdir / f"module.{ext}").write_text(source_code)
        (tmpdir / f"module.test.{ext}").write_text(test_code)
        install = subprocess.run(
            ["npm", "install", "--silent"], cwd=tmpdir, capture_output=True, timeout=90
        )
        if install.returncode != 0:
            return f"npm install failed:\n{install.stderr.decode() or install.stdout.decode()}"
        result = subprocess.run(
            ["npm", "test"], cwd=tmpdir, capture_output=True, text=True, timeout=30
        )
        out = (result.stdout or "") + (result.stderr or "")
        return out if out.strip() else "Tests completed (no output captured)."
    except subprocess.TimeoutExpired:
        return "Test run timed out. Ensure Node.js and npm are installed."
    except FileNotFoundError:
        return "Node.js/npm not found. Install from https://nodejs.org"
    except Exception as e:
        return f"Error: {e}"
    finally:
        shutil.rmtree(tmpdir, ignore_errors=True)

In [10]:
# Sample code for the UI
SAMPLE_JS = '''export function add(a, b) {
  return a + b;
}

export function divide(a, b) {
  if (b === 0) throw new Error("division by zero");
  return a / b;
}

export class Counter {
  constructor(start = 0) {
    this.value = start;
  }
  inc(by = 1) {
    this.value += by;
    return this.value;
  }
}
'''

SAMPLE_TS = '''export function add(a: number, b: number): number {
  return a + b;
}

export function divide(a: number, b: number): number {
  if (b === 0) throw new Error("division by zero");
  return a / b;
}

export class Counter {
  private value: number;
  constructor(start = 0) {
    this.value = start;
  }
  inc(by = 1): number {
    this.value += by;
    return this.value;
  }
}
'''

In [ ]:
with gr.Blocks(title="AI Unit Test Generator (JS/TS)") as ui:
    gr.Markdown("## AI Unit Test Generator (JavaScript & TypeScript)")
    gr.Markdown("Paste JS or TS functions/classes, choose a model, and generate Jest unit tests. Requires Node.js and npm.")

    lang_radio = gr.Radio(choices=["javascript", "typescript"], value="javascript", label="Language")

    with gr.Row():
        code_in = gr.Code(
            label="Source code (export your functions/classes)",
            value=SAMPLE_JS,
            language="javascript",
            lines=18,
        )
        test_out = gr.Code(
            label="Generated test file (Jest)",
            language="javascript",
            lines=24,
        )

    with gr.Row():
        model_dd = gr.Dropdown(choices=models, value=models[0] if models else None, label="Select Model")
        gen_btn = gr.Button("Generate tests", variant="primary")
        run_btn = gr.Button("Run tests", variant="secondary")

    output_txt = gr.Textbox(label="Test output", lines=12, interactive=False)

    def on_lang_change(lang):
        return SAMPLE_TS if lang == "typescript" else SAMPLE_JS

    def on_generate(model, code, lang):
        return generate_unit_tests(model, code, lang)

    def on_run(source, tests, lang):
        return run_tests(source, tests, lang)

    lang_radio.change(fn=on_lang_change, inputs=[lang_radio], outputs=[code_in])
    gen_btn.click(fn=on_generate, inputs=[model_dd, code_in, lang_radio], outputs=[test_out])
    run_btn.click(fn=on_run, inputs=[code_in, test_out, lang_radio], outputs=[output_txt])

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
